<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab5/lab6_BERT_family.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 6 : BERT_family

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.

---


## 목차

1. BERT Fine-tuning with Huggingface
    - Dataset
    - Tokenizer
    - Model
    - Trainer
    - Training
    - Evaluation
2. BERT family
    - RoBERTa
    - DistilBERT
    - SpanBERT
    - BigBird

## 1. BERT Fine-tuning

본 실습에서는 Huggingface의 Transformers 라이브러리를 이용하여 BERT를 fine-tuning하는 과정을 진행한다.

해당 과정을 통해 기본적인 Transformers 라이브러리의 사용법을 익히고, Fine-tuning에 대하여 이해한다.

또한 해당 과정으로부터 HuggingFace의 다양한 모델들을 사용하는 방법을 익힌다.

## 1.1 Dataset

본 실습에서는 [Yelp Review](https://www.yelp.com/dataset) 데이터셋을 사용한다. 해당 Dataset은 `datasets` 라이브러리를 통해 쉽게 불러올 수 있다.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
print(dataset['train'][0])

## 1.2 Tokenizer

Huggingface의 Transformers 라이브러리에서는 다양한 Tokenizer를 제공한다. 본 실습에서는 BERT를 사용하기 때문에 BERT Tokenizer를 사용한다.

AutoTokenizer를 통해 BERT Tokenizer를 불러올 수 있다.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(x):
    return tokenizer(x["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

## 1.3 Model

Huggingface의 Transformers 라이브러리에서는 다양한 pre-trained 모델들을 제공한다.

또한, 해당 모델들은 다양한 Task에 맞게 구조가 변경되어 있다.

본 실습에서는, Sequence Classification Task를 위한 BERT 모델을 불러온다.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [ ]:
print(model.config)

## 1.4 Trainer

Huggingface의 Transformers 라이브러리에서는 Transformer model training에 최적화된 Trainer class를 제공한다.

이를 통해 직접 Training loop를 구현하지 않고도 간단하게 Training을 진행할 수 있다.

하지만 본 실습에서는, 직접 Training loop를 구현하는 방법을 익히기 위해 Trainer class를 사용하지 않고 진행한다.

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

In [ ]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(5000))

### Dataloader

각 train/test set에 대한 DataLoader를 사용하여 Batch 단위로 데이터를 불러온다.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

### Optimizer / Scheduler

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

## 2. BERT family

Huggingface의 Transformers 라이브러리에서는 다양한 BERT family 모델들을 제공한다.

본 실습에서는 다양한 BERT family 모델들을 사용해보고, 각 모델들의 차이점을 이해한다.

[ANSWER]위에서 진행한 Task를 Hugging Face에서 제공하는 다양한 모델에 대해서 진행해보고, 해당 Model의 Configuration을 이해한 뒤, 각 모델들의 차이점을 설명하시오. (마지막 Cell)

### RoBERTa

In [ ]:
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function(x):
    return tokenizer(x["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

#RoBERTa
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=5)

print(model.config)

#DataLoader
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(5000))

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

##Optimizer and Scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

model.to(device)

#Training
progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

#Evaluation
metric = evaluate.load("accuracy")
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

### DistilBERT

### BigBird

[ANSWER] : 최소 3개의 Hugging Face BERT Familly Model에 대해 Task를 진행하고, 각 모델의 특장점에 대해 설명하시오.